In [ ]:
from docx import Document
# 读取word中表格，处理成表头加行的形式
def read_table_from_word(file_path):
    all_rows = []
    doc = Document(file_path)
    for i, table in enumerate(doc.tables):
        for row in table.rows:
            cells_text = [cell.text.replace('\n', '') for cell in row.cells]
            all_rows.append(cells_text)
    new_all_rows=[]
    i = 0
    # 合并跨页的表格行
    while i<len(all_rows):
        if i==len(all_rows)-1:
            new_all_rows.append(all_rows[i])
            break
        if all_rows[i+1][0]!='':
            new_all_rows.append(all_rows[i])
            i+=1
        else:
            new_all_rows.append([a+b for a, b in zip(all_rows[i],all_rows[i+1])])
            i+=2
            
    final_rows=[]
    flag = []
    # 合并表头
    for row in new_all_rows:
        
        if row[0]=='序号':
            flag = row
        merge_now = '|'.join(flag)+'\n' + '|'.join(row)
        final_rows.append(merge_now)

    return final_rows

In [ ]:
from langchain_core.documents import Document as doc
import os
file_path = '/home/user/wyf/rag_table.docx'

def get_docs(file_path):
    text_chunks = read_table_from_word(file_path)
    file_name = os.path.basename(file_path)
    # 加入元数据
    docs = [doc(page_content=chunk, metadata={'source':file_name}) for chunk in text_chunks]
    return docs

In [ ]:
splits = get_docs(file_path)

In [ ]:
print(splits[1].page_content)

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
embeddings = HuggingFaceEmbeddings(model_name='/home/user/wyf/fastchat/bge-large-zh-v1.5', model_kwargs = {'device': 'cuda:1'})
llm = ChatOpenAI(temperature=0, model='/home/user/Downloads/Qwen2-7B-Instruct/', base_url='http://10.250.2.23:8600/v1', api_key='ww')
vectorstore = FAISS.from_documents(splits, embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

In [ ]:
recall_text = retriever.get_relevant_documents('斑小将美白隔离防晒乳的检验结果')
text = ''
for i in recall_text:
    text += i.page_content
    text += '\n\n'

In [ ]:
print(text)

In [ ]:
prompt = '''任务目标：根据给定的知识，回答用户提出的问题。
任务要求：
1、不得脱离给定的知识进行回答。
2、如果给定的知识不包含问题的答案，请回答我不知道。

知识：
{}

用户问题：
{}
'''

In [ ]:
query = '斑小将美白隔离防晒乳的检验结果'
print(llm.invoke(prompt.format(text,query)).content)

In [ ]:
import mammoth
with open("/home/user/wyf/rag_test.docx", "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    html = result.value 
    print(html)

In [ ]:
with open('tmp.html', 'w', encoding='utf-8') as f:
    f.write(html)

In [ ]:
from langchain_text_splitters import HTMLSectionSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLSectionSplitter(headers_to_split_on)

html_header_splits = html_splitter.split_text(html)
for i in html_header_splits:
    print(i)
    print("=======================================================")
# chunk_size = 500
# chunk_overlap = 50
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size, chunk_overlap=chunk_overlap
# )

# # Split
# splits = text_splitter.split_documents(html_header_splits)

In [ ]:
vectorstore = FAISS.from_documents(html_header_splits, embeddings)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k':1})

In [ ]:
recall_text = retriever.get_relevant_documents('国家药监局什么时候发布了《关于更新化妆品禁用原料目录的公告》')
text = ''
for i in recall_text:
    text += i.page_content
    text += '\n\n'

In [ ]:
query = '国家药监局什么时候发布了《关于更新化妆品禁用原料目录的公告》'
print(llm.invoke(prompt.format(text,query)).content)